In [1]:
import pandas as pd
import scipy.stats as st


In [2]:
#Lese die Daten aus dem Google Spreadsheet
df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRmaquJTwdA-ew-ufIcJDbbWaBn1P6eMWO5XOqZgAihzhY0zSfIEvfZDYuuQuin4o2_Zy7KMw5YHzDw/pub?gid=1264034226&single=true&output=csv", header=0, names=["Date", "Personen", "Positive", "Anmerkung"])
df

,Date,Personen,Positive,Anmerkung
0,16.10.2021 14:53:01,3,0,bisher noch kaum gemacht (s.o.)
1,16.10.2021 14:53:59,0,0,Das Screening ist für mich zu kompliziert. Mei...
2,16.10.2021 14:59:57,1,0,NaN
3,16.10.2021 15:00:38,2,0,NaN
4,16.10.2021 15:02:08,1,0,NaN
...,...,...,...,...
72,17.10.2021 22:16:18,0,0,NaN
73,17.10.2021 22:27:09,0,0,NaN
74,17.10.2021 22:50:27,5,0,NaN
75,17.10.2021 23:20:24,3,0,NaN


In [3]:
#Daten bereinigen
#Hepatitis E-Fall abziehen
df.at[5,['Positive']]=0

In [4]:
#Errechne die Summe der Personen
anzahl = df["Personen"].sum()
anzahl

352

In [5]:
#Zähle die Positiven
positive = df["Positive"].sum()
positive

3

In [6]:
#Errechne die Positivenquote in Prozent
quote = positive / anzahl *100
quote

0.8522727272727272

In [7]:
#Anzahl der Praxen ohne bisher gescreent zu haben:
ohne_screening = df["Personen"].value_counts()
anz_teilnehmende = len(df.index)
print(anz_teilnehmende)
ohne_screening[0]

77


36

In [8]:
#Anteil der Teilnehmenden ohne Screening bisher in Prozent
anteil_nichtscreener = ohne_screening[0]/anz_teilnehmende*100
print(anteil_nichtscreener)

46.75324675324675


In [9]:
#Erzeuge neuen Datenframe mit Teilnehmenden über 0 Personen

df_teilnehmende = df.loc[df["Personen"] > 0]
df_teilnehmende.head()


,Date,Personen,Positive,Anmerkung
0,16.10.2021 14:53:01,3,0,bisher noch kaum gemacht (s.o.)
2,16.10.2021 14:59:57,1,0,NaN
3,16.10.2021 15:00:38,2,0,NaN
4,16.10.2021 15:02:08,1,0,NaN
5,16.10.2021 15:19:36,10,0,Hepatitis E


In [10]:
# Spalte Anmerkung löschen
df_teilnehmende.drop(columns=["Anmerkung"],axis=1,inplace=True)
df_teilnehmende.head()


c:\Users\slodd\OneDrive\Dokumente\prj\trader\.venv\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Date,Personen,Positive
0,16.10.2021 14:53:01,3,0
2,16.10.2021 14:59:57,1,0
3,16.10.2021 15:00:38,2,0
4,16.10.2021 15:02:08,1,0
5,16.10.2021 15:19:36,10,0


In [11]:
#Positivenrate per Praxis berechnen (erfordlich für CI)
#df_teilnehmende["Rate"] = df_teilnehmende['Positive']/df_teilnehmende['Personen']
#df_teilnehmende
df_teilnehmende.insert(loc=len(df_teilnehmende.columns), column='Rate', value=df_teilnehmende.Positive/df_teilnehmende.Personen)
df_teilnehmende.tail()


,Date,Personen,Positive,Rate
0,16.10.2021 14:53:01,3,0,0.0
2,16.10.2021 14:59:57,1,0,0.0
3,16.10.2021 15:00:38,2,0,0.0
4,16.10.2021 15:02:08,1,0,0.0
5,16.10.2021 15:19:36,10,0,0.0


In [12]:
#Konfidenzintervall der Positivenrate berechnen
#st.norm.interval(alpha=0.95, loc=df_teilnehmende['Rate'].mean(), scale=st.sem(df_teilnehmende['Rate']))
ci = st.t.interval(alpha=0.95, df=len(df_teilnehmende['Rate'])-1, loc=df_teilnehmende['Rate'].mean(), scale=st.sem(df_teilnehmende['Rate'])) 
print(f"Konfidenzintervall: {ci[0]*100:.2f} bis {ci[1]*100:.2f}%")

Konfidenzintervall: -0.31 bis 1.76%
